# Working with Census Data Using Pandas

## Matthew Leick-Macari

## Omaha Python User Group
## June 2019

## Agenda/Topics:

* What is the American Community Surey (ACS)?
* How to get data for the ACS?
* What is pandas? 
* How to work with ACS data using Pandas?

# What is the American Community Survey (ACS)?

> The ACS is an ongoing surevy that provides vital information on a yearly basis about our nation and its people

* The ACS is based on a randomly selected group of households that are surveyed annually.
* The Census Bureau is prohibited by fedeeral law from sharing individual respondent data with anyone - not the IRS, the FBI, CIA, etc.  They are also legally obligated to expunge respondent level demographic data.


### References
* [American Community Survey (ACS)](https://www.census.gov/programs-surveys/acs)

## What kind of questions are on the ACS?

There are five (5) questions pertaining to where a respondent lives and how they get to work.

* Questions pretain to how someone got to work last week, who they went with, what time they left to go to work, and how many minutes it took to get to work.
* This data can be used for transportation planning - local, state, and federal agencies use these statistics to plan programs and services for disabled populations, bicycle commuters, carpools and ride shares.
* Understanding the commuting patterns is important for defining banking and housing markets.
* Coffee shops might want to understand where people work and how they get there when planning a new cafe.


### Source

* [Why we ask questions about Commuting/Journey to Work](https://www.census.gov/acs/www/about/why-we-ask-each-question/commuting/)

## Some reasults...

### United States:    

* _Average Commute Time_ - 26.4 miutes
* _Workers Who Used Public Transportation_ - 5.1%
* _Workers Who Worked at Home_ - 4.7%

### Nebrasks

* _Average Commute Time_ - 18.4 miutes
* _Workers Who Used Public Transportation_ - 0.7%
* _Workers Who Worked at Home_ - 4.3%
    
### Omaha

* _Average Commute Time_ - 18.7 miutes
* _Workers Who Used Public Transportation_ - 1.4%
* _Workers Who Worked at Home_ - 3.6%  

## How do we get information from the ACS?


* ACS data is available via flat files, and an API
* To use the API, you'll need to get an API key.
* The Census Bureau has worked to make the ACS API machine readable.

## Requests: HTTP for Humans

#### _Get Requests:_

```sh
pip install requests
```

#### _In the REPL:_
```python
>>> import requests
>>> r = requests.get("https://api.github.com/events")
```

### Source:
* (Requests: HTTP for Humans)[https://2.python-requests.org/en/master/]

In [1]:
import requests

r = requests.get("https://api.github.com/events")

data = r.json()

data[0]

{'id': '9861039106',
 'type': 'PushEvent',
 'actor': {'id': 45344090,
  'login': '2017liqi',
  'display_login': '2017liqi',
  'gravatar_id': '',
  'url': 'https://api.github.com/users/2017liqi',
  'avatar_url': 'https://avatars.githubusercontent.com/u/45344090?'},
 'repo': {'id': 181891121,
  'name': '2017liqi/jiaokao',
  'url': 'https://api.github.com/repos/2017liqi/jiaokao'},
 'payload': {'push_id': 3735421964,
  'size': 1,
  'distinct_size': 1,
  'ref': 'refs/heads/master',
  'head': 'afb91b4f0a34272de805c1174eff5d96e55d8730',
  'before': 'bbdb455643528d7c640a24374b669a0cb0fe9ac3',
  'commits': [{'sha': 'afb91b4f0a34272de805c1174eff5d96e55d8730',
    'author': {'email': '1833718739@qq.com', 'name': '1833718739'},
    'message': '界面调整',
    'distinct': True,
    'url': 'https://api.github.com/repos/2017liqi/jiaokao/commits/afb91b4f0a34272de805c1174eff5d96e55d8730'}]},
 'public': True,
 'created_at': '2019-06-20T13:43:57Z'}

## Extracting Data from the Census API

### Anatomy of a Variable

* Name: B08302_001E
* Label: Estimate!!Total
* Concept: TIME LEAVING HOME TO GO TO WORK
* Group: B08302

| Type of Variable              | Sample Variable Name | Suffix |
|-------------------------------|----------------------|--------|
| Estimate                      | B08302_001E          | E      |
| Annotation of Estimate        | B08302_001EA         | EA     |
| Margin of Error               | B08302_001M          | M      |
| Annotation of Margin of Error | B08302_001MA         | MA     |

In [2]:
import os
import requests as r

# Retrieve API key
API_KEY = os.environ.get("CENSUS_API_KEY")

URL_STUB = "https://api.census.gov/data/2017/acs/acs5?get={variable}&for={level_of_detail}:*&in=state:{fips_state}&key={api_key}"

# https://api.census.gov/data/2017/acs/acs5/groups/B08302.json

variable = "B08302_007E" # Estimate!!Total!!7 00 a.m. to 7 29 a.m.
fips_state = "31"

api_url = URL_STUB.format(
    variable=variable, 
    fips_state=fips_state, 
    api_key=API_KEY, 
    level_of_detail='tract'
)

api_resp = r.get(api_url)

json_data = api_resp.json()
json_data[0:10]


[['B08302_007E', 'state', 'county', 'tract'],
 ['155', '31', '079', '000200'],
 ['455', '31', '079', '000600'],
 ['422', '31', '079', '000500'],
 ['350', '31', '079', '000900'],
 ['340', '31', '079', '000800'],
 ['246', '31', '079', '001300'],
 ['142', '31', '079', '001200'],
 ['136', '31', '079', '001400'],
 ['59', '31', '085', '961500']]

## Pandas: 101

Pandas is a data manipulation library that is part of the pydata ecosystem.

_Get Pandas:_

Via conda:
```sh
conda install pandas
```

Via pypi:
```sh
pip install --upgrade pandas
```

### Sources:
* [Pandas](http://pandas.pydata.org)
* [Pandas Docs](http://pandas.pydata.org/pandas-docs/version/0.24/pandas.pdf)

In [3]:
import pandas as pd
import numpy as np

# cleanup to get the list into a dict

header = json_data[0]

data_set = [dict(zip(header, elem)) for elem in json_data[1:]]

# Using the data from the previous slide, let's build a data frame

df = pd.DataFrame(data_set, columns=['state', 'county', 'tract', 'B08302_007E'])

df.head(5)

,state,county,tract,B08302_007E
0,31,079,000200,155
1,31,079,000600,455
2,31,079,000500,422
3,31,079,000900,350
4,31,079,000800,340


### Pandas Data Structures


* Core data strutures are:
    
    * 1 dimensional series -> vectors
    * 2 dimensional data frames -> matrix
    * N-dimensional panels -> tensor
    
* A useful mental model for data frames is to think of as analogus to database tables.

In [4]:
# Filtering operations

# Adams == 001
df[df['county'] == '001']

,state,county,tract,B08302_007E
49,31,001,965600,314
50,31,001,965800,66
51,31,001,966200,336
52,31,001,966100,91
53,31,001,965500,314
54,31,001,965900,459
55,31,001,966000,150
56,31,001,965400,397
57,31,001,965700,73


In [5]:
# We can also create new variables, and delete old ones.

df['fips_code'] = df['state'] + df['county']

del df['county']

df = df[['fips_code', 'tract', 'B08302_007E']]

# Casting to the correct types is important...
df['B08302_007E'] = df['B08302_007E'].astype('int64')

df.head(5)

,fips_code,tract,B08302_007E
0,31079,000200,155
1,31079,000600,455
2,31079,000500,422
3,31079,000900,350
4,31079,000800,340


In [6]:
grouped_df = df.groupby(['fips_code'])

grouped_df = grouped_df.agg([np.size, np.sum, np.mean, np.median, np.std])

grouped_df.sort_values(by=('B08302_007E', 'size'), ascending=False).head(10)

B08302_007E                                      
                 size    sum        mean median         std
fips_code                                                  
31055             156  45820  293.717949  256.5  177.196585
31109              74  24393  329.635135  325.5  195.823206
31153              43  18456  429.209302  371.0  206.138021
31079              14   3612  258.000000  244.0  108.503102
31157              11   2343  213.000000  212.0   77.939720
31019              11   3033  275.727273  297.0  132.336005
31001               9   2200  244.444444  314.0  150.443603
31119               9   2372  263.555556  253.0  115.315991
31053               9   2866  318.444444  325.0  131.783071
31111               8   2397  299.625000  253.0  129.054736

## What's next?

We need to clean up the data frame, and make it presentable.

In [7]:
# we care about the top 10 counties.
top_10_df = grouped_df.sort_values(by=('B08302_007E', 'size'), ascending=False).head(10)

top_10_df = top_10_df['B08302_007E'].reset_index()

top_10_df

,fips_code,size,sum,mean,median,std
0,31055,156,45820,293.717949,256.5,177.196585
1,31109,74,24393,329.635135,325.5,195.823206
2,31153,43,18456,429.209302,371.0,206.138021
3,31079,14,3612,258.000000,244.0,108.503102
4,31157,11,2343,213.000000,212.0,77.939720
5,31019,11,3033,275.727273,297.0,132.336005
6,31001,9,2200,244.444444,314.0,150.443603
7,31119,9,2372,263.555556,253.0,115.315991
8,31053,9,2866,318.444444,325.0,131.783071
9,31111,8,2397,299.625000,253.0,129.054736


In [8]:
# Let's rename the columns

column_mapping = {
    'size': 'tract_count',
    'sum': 'total'
}

top_10_df.rename(columns=column_mapping, inplace=True)

top_10_df

,fips_code,tract_count,total,mean,median,std
0,31055,156,45820,293.717949,256.5,177.196585
1,31109,74,24393,329.635135,325.5,195.823206
2,31153,43,18456,429.209302,371.0,206.138021
3,31079,14,3612,258.000000,244.0,108.503102
4,31157,11,2343,213.000000,212.0,77.939720
5,31019,11,3033,275.727273,297.0,132.336005
6,31001,9,2200,244.444444,314.0,150.443603
7,31119,9,2372,263.555556,253.0,115.315991
8,31053,9,2866,318.444444,325.0,131.783071
9,31111,8,2397,299.625000,253.0,129.054736


In [10]:
# let's make the fips code column more intelligble...

# the notebook is executed in the folder where it's located, this data is in the parent directory
county_file_path = '../ref_data/county_list.csv'

county_df = pd.read_csv(county_file_path)

# type casting is important, because pandas tries to be "helpful"
county_df['fips_code'] = county_df['fips_code'].astype('str')

county_df.head(10)

,fips_code,county,state_abbrv
0,31001,Adams,NE
1,31003,Antelope,NE
2,31005,Arthur,NE
3,31007,Banner,NE
4,31009,Blaine,NE
5,31011,Boone,NE
6,31013,Box Butte,NE
7,31015,Boyd,NE
8,31017,Brown,NE
9,31019,Buffalo,NE


In [11]:
# Luckily, we can merge dataframes...

top_10_df_pretty = top_10_df.merge(county_df, on='fips_code')
top_10_df_pretty = top_10_df_pretty[['county', 'fips_code', 'tract_count', 'total', 'median', 'mean', 'std']]

top_10_df_pretty

,county,fips_code,tract_count,total,median,mean,std
0,Douglas,31055,156,45820,256.5,293.717949,177.196585
1,Lancaster,31109,74,24393,325.5,329.635135,195.823206
2,Sarpy,31153,43,18456,371.0,429.209302,206.138021
3,Hall,31079,14,3612,244.0,258.000000,108.503102
4,Scotts Bluff,31157,11,2343,212.0,213.000000,77.939720
5,Buffalo,31019,11,3033,297.0,275.727273,132.336005
6,Adams,31001,9,2200,314.0,244.444444,150.443603
7,Madison,31119,9,2372,253.0,263.555556,115.315991
8,Dodge,31053,9,2866,325.0,318.444444,131.783071
9,Lincoln,31111,8,2397,253.0,299.625000,129.054736
